# Loading necessary libraries

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.initializers import  RandomNormal
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
%matplotlib inline
from tensorflow.keras.preprocessing import image

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import pandas as pd

# Checking if GPU is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Loading and preprocessing images

In [6]:
emotions_names = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
train_data_dir = "../fer2013/train"
test_data_dir = "../fer2013/test"

In [7]:
# Initialize lists to store images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []

In [10]:
# Load training images and labels
for emotion_folder in os.listdir(train_data_dir):
    emotion_label = emotion_folder.lower()  # Ambil nama folder dan ubah menjadi huruf kecil
    emotion_id = emotions_names.get(emotion_label, -1)
    if emotion_id != -1:
        emotion_folder_path = os.path.join(train_data_dir, emotion_folder)
        for image_file in os.listdir(emotion_folder_path):
            image_path = os.path.join(emotion_folder_path, image_file)
            image = Image.open(image_path)
            image = image.convert('L')  # Convert to grayscale
            image = image.resize((48, 48))  # Resize the image to your desired size
            image_data = np.array(image)
            train_images.append(image_data)
            train_labels.append(emotion_id)


In [12]:
# Load training images and labels
for emotion_folder in os.listdir(test_data_dir):
    emotion_label = emotion_folder.lower()  # Ambil nama folder dan ubah menjadi huruf kecil
    emotion_id = emotions_names.get(emotion_label, -1)
    if emotion_id != -1:
        emotion_folder_path = os.path.join(train_data_dir, emotion_folder)
        for image_file in os.listdir(emotion_folder_path):
            image_path = os.path.join(emotion_folder_path, image_file)
            image = Image.open(image_path)
            image = image.convert('L')  # Convert to grayscale
            image = image.resize((48, 48))  # Resize the image to your desired size
            image_data = np.array(image)
            train_images.append(image_data)
            train_labels.append(emotion_id)


In [14]:
# Convert lists to numpy arrays
X_train = np.array(train_images)
y_train = np.array(train_labels)
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Normalize pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert emotion labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=len(emotions_names))
y_test = to_categorical(y_test, num_classes=len(emotions_names))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# Building CNN

In [7]:
# Constructing CNN structure

# VGG16 arsitektur
# Membuat model VGG16 dengan citra grayscale
model = Sequential()

# Layer 1 (grayscale input)
model.add(Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 1000 classes for ImageNet

# Compile model


In [8]:
# Tampilkan ringkasan model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                    

# Training CNN

In [9]:
# Compiling model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])



# Specifying parameters for Data Augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    zoom_range = 0.05)  # zoom images in range [1 - zoom_range, 1+ zoom_range]

datagen.fit(X_train)

history = model.fit_generator(datagen.flow(X_train, y_train,
                    batch_size=32),
                    epochs=20,  # Mengganti nb_epoch menjadi epochs
                    validation_data=(X_val, y_val),
                    steps_per_epoch=X_train.shape[0] // 32,  # Menggunakan // untuk pembagian bulat
                    )



pd.DataFrame(history.history).to_csv("history.csv")

Epoch 1/20


C:\Users\hp\AppData\Local\Temp\ipykernel_10260\3430552048.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train,


908/908 [==============================] - ETA: 0s - loss: 1.8172 - accuracy: 0.2483
Epoch 1: val_loss improved from inf to 1.81802, saving model to Model.hdf5
908/908 [==============================] - 47s 47ms/step - loss: 1.8172 - accuracy: 0.2483 - val_loss: 1.8180 - val_accuracy: 0.2492
Epoch 2/20
907/908 [============================>.] - ETA: 0s - loss: 1.8135 - accuracy: 0.2513
Epoch 2: val_loss improved from 1.81802 to 1.80980, saving model to Model.hdf5
908/908 [==============================] - 41s 46ms/step - loss: 1.8135 - accuracy: 0.2513 - val_loss: 1.8098 - val_accuracy: 0.2492
Epoch 3/20
908/908 [==============================] - ETA: 0s - loss: 1.8127 - accuracy: 0.2514
Epoch 3: val_loss improved from 1.80980 to 1.80924, saving model to Model.hdf5
908/908 [==============================] - 42s 46ms/step - loss: 1.8127 - accuracy: 0.2514 - val_loss: 1.8092 - val_accuracy: 0.2492
Epoch 4/20
908/908 [==============================] - ETA: 0s - loss: 1.8124 - accuracy: 0.

KeyboardInterrupt: 

In [ ]:
model.save('VGG20epoch.h5')

# Loading best model and exploring the results

In [ ]:
model2 = load_model('VGG20epoch.h5')

In [ ]:
model.evaluate(np.array(X_train), y_train)

In [ ]:
model2.evaluate(np.array(X_train), y_train)

In [ ]:
test_evaluation = model.evaluate(np.array(X_test), y_test)
train_evaluation = model.evaluate(np.array(X_train), y_train)

In [ ]:
test_evaluation = model2.evaluate(np.array(X_test), y_test)
train_evaluation = model2.evaluate(np.array(X_train), y_train)

In [ ]:
history = pd.read_csv('VGG.csv')

In [ ]:
# grafik salah
def plot_accuracy(data, size=(20, 10)):
    plt.figure(figsize=size) 
    plt.plot(data['accuracy'])  # Menggunakan 'accuracy'
    plt.plot(data['val_accuracy'])  # Menggunakan 'val_accuracy'
    plt.title('Model Accuracy', fontsize=18)
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel('Accuracy', fontsize=18)
    plt.legend(['Train', 'Test'], loc='upper left', fontsize=16)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()

plot_accuracy(history)

In [ ]:
# salah
def plot_loss(data, size = (20,10)):
    plt.figure(figsize=size) 
    plt.plot(data['loss'])
    plt.plot(data['val_loss'])
    plt.title('Model Loss', fontsize = 18)
    plt.ylabel('Loss', fontsize = 18)
    plt.xlabel('Epoch', fontsize = 18)
    plt.ylim(0.9,2)
    plt.legend(['Train', 'Test'], loc='upper left', fontsize = 16)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()
    
plot_loss(history)

In [ ]:
def predict_classes(model, test_imgs, test_labels, emotions_dict,  batch_size  = 32):    

    # Predict class of image using trained model
    class_pred = model.predict(test_imgs, batch_size = batch_size)

    # Convert vector of zeros and ones to label
    labels_pred = np.argmax(class_pred,axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    # Boolean array that indicates whether the predicted label is the true label
    correct = labels_pred == true_labels
    
    # Converting array of labels into emotion names
    pred_emotion_names = pd.Series(labels_pred).map(emotions_dict)
    
    results = {'Predicted_label': labels_pred, 'Predicted_emotion': pred_emotion_names, 'Is_correct' : correct}
    results = pd.DataFrame(results)
    return correct, results


def visualize_predictions(images_test, orglabel_names, predlabel_names, correct_arr, valid = True):
    
    if valid == True:
        correct = np.array(np.where(correct_arr == True))[0]
        # Plot 15 randomly selected and correctly predicted images
        show_random(images_test, emotion_nms_org = orglabel_names, emotion_nms_pred = predlabel_names, random = False, indices = correct)
    else:
        incorrect = np.array(np.where(correct_arr == False))[0]
        # Plot 15 randomly selected and wrongly predicted images
        show_random(images_test, emotion_nms_org = orglabel_names, emotion_nms_pred = predlabel_names, random = False, indices = incorrect)
 

In [ ]:
correct, results_df = predict_classes(model, X_test, y_test, emotions_names, batch_size = 64)
results_df['Original_label'] = data['emotion'][32298:].values
results_df['True_emotion'] = results_df['Original_label'].map(emotions_names)

### Correctly predicted images

In [ ]:
def visualize_predictions(images, true_emotions, predicted_emotions, correct, valid=True):
    plt.figure(figsize=(12, 12))
    count = 0
    
    for i in range(len(images)):
        if correct[i] == valid:  # Filter based on whether prediction is correct or not
            count += 1
            # Reshape the flattened image to (48, 48)
            reshaped_image = images[i].reshape(48, 48)
            plt.subplot(5, 5, count)
            plt.imshow(reshaped_image, cmap='gray')
            plt.title(f'True: {true_emotions[i]}\nPredicted: {predicted_emotions[i]}')
            plt.axis('off')
            
            if count >= 25:  # Display up to 25 images
                break
    
    plt.show()

# Call the function to visualize incorrect predictions
visualize_predictions(images_test, results_df['True_emotion'], results_df['Predicted_emotion'], correct, valid=True)


### Wronly predicted images

In [ ]:
def visualize_predictions(images, true_emotions, predicted_emotions, correct, valid=True):
    plt.figure(figsize=(12, 12))
    count = 0
    
    for i in range(len(images)):
        if correct[i] == valid:  # Filter based on whether prediction is correct or not
            count += 1
            # Reshape the flattened image to (48, 48)
            reshaped_image = images[i].reshape(48, 48)
            plt.subplot(5, 5, count)
            plt.imshow(reshaped_image, cmap='gray')
            plt.title(f'True: {true_emotions[i]}\nPredicted: {predicted_emotions[i]}')
            plt.axis('off')
            
            if count >= 25:  # Display up to 25 images
                break
    
    plt.show()

# Call the function to visualize incorrect predictions
visualize_predictions(images_test, results_df['True_emotion'], results_df['Predicted_emotion'], correct, valid=False)


### Confusion matrix

In [ ]:
def create_confmat(true_labels, predicted_labels, columns, colour = 'Oranges', size = (20,14)):
    
    cm = confusion_matrix(true_labels, predicted_labels) 
    cm_df = pd.DataFrame(cm,
    index = [col for col in columns], 
    columns = [col for col in columns])
    plt.figure(figsize=(18,16))
    sns.heatmap(cm_df, annot = True, cmap = colour, fmt='g', linewidths=.2)
    plt.title('Confusion Matrix', fontsize = 20)
    plt.ylabel('True label', fontsize = 18)
    plt.xlabel('Predicted label', fontsize = 18)
    plt.tick_params(axis='both', labelsize=14)
    plt.show()

In [ ]:
# Displaying confusion matrix
create_confmat(results_df['Original_label'], results_df['Predicted_label'], ['Angry','Disgust','Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], colour = 'Greens')

In [ ]:
# Path ke gambar
img_path = 'D:/Semester_7/FER-2013/test/fear/PrivateTest_2632530.jpg'

# Memuat gambar ke dalam bentuk grayscale
test_image = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

# Melakukan prediksi
result = model.predict(test_image)
predicted_class = np.argmax(result[0])

emotions_names = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
print(emotions_names)

# Menampilkan gambar
plt.imshow(test_image[0, :, :, 0], cmap='gray')  # Munculkan gambar grayscale
plt.title('Predicted class: {}'.format(predicted_class))
plt.show()